IMPORT LIBRARIES

In [ ]:
# Import libraries
import os
import librosa
import csv
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema
import sounddevice as sd

TASK 1: Read wav files and create numpy array of them

    TASK 1.1 Get voiced parts

In [ ]:
def get_auto_correlation(clip, FRAME_SIZE = 100):
    bins = np.zeros(FRAME_SIZE)
    for i in range(0, FRAME_SIZE):
        for j in range(0, FRAME_SIZE-i):
            bins[i] += (clip[j] * clip[j+i])
    return bins

#doesnt work perfectly - it doesnt connect all parts of audio
def get_voiced_parts(clip, sr, FRAME_SIZE = 100):
    IS_CURR_VOICED = False

    MAX_NUM_FRAMES = 25
    START_NUM_FRAMES = 20
    MIN_NUM_OF_SAMPLES_IN_SEGMENT = 0.2 * sr
    MIN_RMS = 0.05

    voiced_counter = 0
    start_of_segment = -1
    end_of_segment = -1
    end_of_checking = (len(clip)//FRAME_SIZE-1) * FRAME_SIZE

    for i in range(0, end_of_checking, FRAME_SIZE):
        #for each frame
        bins = np.zeros(FRAME_SIZE)
        bins = get_auto_correlation(clip[i:i+FRAME_SIZE], FRAME_SIZE)
        #plt.stem(bins)
        maximums = np.diff(argrelextrema(bins, np.greater))
        median_maximums = np.median(maximums)
        #if median maximus in some boudaries
        if median_maximums >= 7 and median_maximums <= 20 and np.std(maximums) < 7:
            voiced_counter = min(voiced_counter + 1, MAX_NUM_FRAMES)

            if not IS_CURR_VOICED:
                voiced_counter = START_NUM_FRAMES
                IS_CURR_VOICED = True
                start_of_segment = i
        else:
            voiced_counter = max(0, voiced_counter - 1)

            if voiced_counter == 0 and IS_CURR_VOICED:
                IS_CURR_VOICED = False
                end_of_segment = i
                #if too short
                if end_of_segment - start_of_segment < MIN_NUM_OF_SAMPLES_IN_SEGMENT:
                    continue 
                rms = np.sqrt(np.mean(clip[start_of_segment:end_of_segment]**2))
                if rms < MIN_RMS:
                    continue
                return clip[start_of_segment: end_of_segment]

    end_of_segment = len(clip)
    
    if IS_CURR_VOICED and ((end_of_segment - start_of_segment) > MIN_NUM_OF_SAMPLES_IN_SEGMENT):
        return clip[start_of_segment: end_of_segment]       
    return None

    

In [ ]:
def normalize(sample):
    sample = sample / np.max(np.abs(sample))
    return sample

TESTING !!!!

In [ ]:
#trying to detect spoken parts with outocorrleation
new_sr = 8000
clip, sr = librosa.load("../data/divide/anja_3.wav")
clip = librosa.resample(clip, orig_sr=sr, target_sr=new_sr)
clip = librosa.to_mono(clip)
clip = normalize(clip)

IS_CURR_VOICED = False

MAX_NUM_FRAMES = 25
START_NUM_FRAMES = 20
MIN_NUM_OF_SAMPLES_IN_SEGMENT = 0.2 * new_sr
MIN_RMS = 0.05

voiced_counter = 0
start_of_segment = -1
end_of_segment = -1
FRAME_SIZE = 100
len_clip_in_frames = len(clip) // FRAME_SIZE - 1
end_of_checking = len_clip_in_frames * FRAME_SIZE
out = np.zeros(len_clip_in_frames)
curr_voiced_over_time = np.zeros(len_clip_in_frames)

for i in range(0, end_of_checking, FRAME_SIZE):
    #for each frame
    bins = np.zeros(FRAME_SIZE)
    bins = get_auto_correlation(clip[i:i+FRAME_SIZE], FRAME_SIZE)
    #plt.stem(bins)
    maximums = np.diff(argrelextrema(bins, np.greater))
    median_maximums = np.median(maximums)
    #if median maximus in some boudaries
    if median_maximums >= 7 and median_maximums <= 20 and np.std(maximums) < 7:
        out[i//FRAME_SIZE] = 1
        voiced_counter = min(voiced_counter + 1,MAX_NUM_FRAMES)
        if  not IS_CURR_VOICED:
            voiced_counter = START_NUM_FRAMES
            IS_CURR_VOICED = True
            start_of_segment = i
    else:
        voiced_counter = max(0, voiced_counter-1)

        if voiced_counter == 0 and IS_CURR_VOICED:
            IS_CURR_VOICED = False
            end_of_segment = i
            #if too short
            if end_of_segment - start_of_segment < MIN_NUM_OF_SAMPLES_IN_SEGMENT:
                continue

            rms = np.sqrt(np.mean(clip[start_of_segment:end_of_segment]**2))
            if rms < MIN_RMS:
                continue

            a = clip[start_of_segment: end_of_segment]
            print(start_of_segment/new_sr, end_of_segment/new_sr)
            plt.axvline(start_of_segment, color='g')
            plt.axvline(end_of_segment, color='g')

    #plt.show() 

    curr_voiced_over_time[i//FRAME_SIZE] = voiced_counter
end_of_segment = len(clip)
if IS_CURR_VOICED and ((end_of_segment - start_of_segment) > MIN_NUM_OF_SAMPLES_IN_SEGMENT):
    plt.axvline(start_of_segment, color='g')
    plt.axvline(end_of_segment, color='g')
    print(end_of_segment - start_of_segment)
#print(start_of_segment/new_sr, end_of_segment/new_sr)
plt.stem(np.repeat(out, FRAME_SIZE)*0.5, 'r')
plt.plot(clip)

plt.plot(np.repeat(curr_voiced_over_time, FRAME_SIZE)*0.1, 'k')
sd.play(a, 8000)
#print(maximums)
#plt.plot(clip)

    TASK 1.2 Normalize between -1 and 1

TASK 2. FEATURE EXTRACTION

    TASK 2.1 Preemphasis filter

In [ ]:
def add_preemphasis_filter(x, coeff=0.95):
    # TODO - on our own??
    x = librosa.effects.preemphasis(x, coef = coeff)
    return x

    TASK 2.2 Create MFCC of samles of uniform size (30, 80)

In [ ]:
def get_2d_features(y, sr = 8000, wanted_width = 40):
    y = add_preemphasis_filter(y)
    #20 row mfcc, 20 row delta mfcc, 1 row pitch...
    windows_size = len(y) // (wanted_width - 1)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mels=20, n_fft=300, hop_length=windows_size, window='hamming')
    delta_mfcc = librosa.feature.delta(mfcc)
    mfcc = np.concatenate((mfcc, delta_mfcc, ), axis=0)
    return mfcc

In [ ]:
def get_1d_features(y, sr = 8000, wanted_width = 5):
    windows_size = len(y)//(wanted_width - 1)
    #coeffients of polynomial fitting frequency information over time
    polly_coeff = librosa.feature.poly_features(y=y, sr=sr, order=0, n_fft=1024, hop_length=windows_size, window='hamming')[0]
    #how many times do signal cross zero
    zero_cross = librosa.feature.zero_crossing_rate(y=y, frame_length=1024, hop_length=windows_size)[0]
    #tone vs noise level in each windows
    tone_vs_noise = librosa.feature.spectral_flatness(y=y, n_fft=1024, hop_length=windows_size)[0]
    
    return np.concatenate((polly_coeff, zero_cross, tone_vs_noise), axis=0)

        TESTING!

In [ ]:
new_sr = 8000
clip, sr = librosa.load("../data/eight/luka_3.wav")
clip = librosa.resample(clip, orig_sr=sr, target_sr=new_sr)
clip = librosa.to_mono(clip)
clip = normalize(clip)
clip = get_voiced_parts(clip, new_sr)
clip = add_preemphasis_filter(clip)
features = get_2d_features(clip, new_sr)


print(get_1d_features(clip))

sd.play(clip, new_sr)
sd.wait()
plt.plot(clip)
plt.show()
plt.imshow(features, cmap='hot', interpolation='nearest')

    TASK 2.3 Other feutures -TODO

Here we see the example main function to create train/validate and test set

In [ ]:
def make_train_data(directory, training_data_name = "training_data.npy"):
    training_data = []
    eye_in = 0
    NUM_CATEGORIES = len(os.listdir(directory))
    #get al subdirctories in directory
    for category in os.listdir(directory):
        print(category)
        CATEGORIES.append(category)
        path = os.path.join(directory, category)
        for file in os.listdir(path):
            try:
                #Fs, audio = wavfile.read(os.path.join(path, file))
                clip, sr = librosa.load(os.path.join(path, file))
                clip = librosa.resample(clip, orig_sr=sr, target_sr=8000)
                clip = librosa.to_mono(clip)

                clip = normalize(clip)
                clip = get_voiced_parts(clip,8000)

                if(clip is None):
                    print("{} not good".format(os.path.join(path, file)))
                    continue

                features_2d = get_2d_features(clip)
                features_1d = get_1d_features(clip)

                training_data.append([features_2d, features_1d, np.eye(NUM_CATEGORIES)[eye_in]])
            except Exception as e:
                print(e)
                pass
        eye_in += 1
    np.random.shuffle(training_data)
    return np.array(training_data, dtype=object)

CATEGORIES = []
training_data = make_train_data(os.path.join(os.getcwd(), "..", "data"))

np.save("training_data.npy", training_data, allow_pickle=True)
print(len(training_data))

In [ ]:
INDEX = 2
plt.imshow(training_data[INDEX][0])
print(training_data[INDEX][1])
print(training_data[INDEX][2])

Task 3. Neural network

In [ ]:
#loading data for training
training_data = np.load("training_data.npy", allow_pickle=True)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the missing variables
height = 0
width = 0

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 16)
        self.conv2 = nn.Conv2d(32, 64, 4)
        self.conv3 = nn.Conv2d(64, 128, 8)
        self.norm3 = nn.BatchNorm2d(128)
        x = torch.randn(height, width).view(-1,1, height, width)
        self._to_linear=None
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, len(CATEGORIES))

    def convs(self,x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, (2,2)) #pooling space 2x2
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, (2,2)) #pooling space 2x2
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, (2,2)) #pooling space 2x2
        x = self.norm3(x)

        if self._to_linear is None:
            self._to_linear = x[0].shape[0] * x[0].shape[1] * x[0].shape[2]
        return x
    
    def forward(self,x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x,dim=1)

# Initialize on GPU
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("GPU")
else:
    device = torch.device("cpu")
    print("CPU")

net = Net().to(device)

In [ ]:
import torch.optim as optim

#optimizer
optimizer = optim.Adam(net.parameters(), lr = 0.001) #learning rate
napaka_f = nn.MSELoss()

train_X = torch.Tensor([i[0] for i in training_data]).view(-1, height, width) #flatten
#X = X/255.0 #normaliziram
train_y = torch.Tensor([i[1] for i in training_data])

In [ ]:
from tqdm import tqdm
from random import shuffle

#training
BATCH_SIZE = 8
EPOH = 3000

for epoch in tqdm(range(EPOH)):
    for i in range(0, len(train_X), BATCH_SIZE):
        batch_X = train_X[i:i+BATCH_SIZE].view(-1,1,height,width)
        batch_y = train_y[i:i+BATCH_SIZE]

        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        net.zero_grad() #damo zero gradiant
        out = net(batch_X)
        napaka = napaka_f(out, batch_y)
        napaka.backward()
        optimizer.step()
    train_X, train_y = shuffle(train_X,train_y)
print(napaka)


# Record new data

In [ ]:
import os
import sounddevice as sd
from scipy.io import wavfile 

freq = 44100
duration = 4

print("Recording: start speaking...")
recording = sd.rec(int(duration * freq), samplerate=freq, channels=2)

sd.wait()

filepath = os.path.join(os.getcwd(), "..", "data", "input", "input.wav")
wavfile.write(filepath, freq, recording)  # Fix the function call



In [ ]:
# Read new data and classify
clip, sr = librosa.load(filepath)
clip = librosa.resample(clip, orig_sr=sr, target_sr=8000)
clip = librosa.to_mono(clip)
clip = normalize(clip)
clip = get_voiced_parts(clip,8000)
clip = add_preemphasis_filter(clip)
features = get_2d_features(clip, 8000)
features = torch.Tensor(features).view(-1,1,height,width)
features = features.to(device)
out = net(features)
print(out)